<a href="https://colab.research.google.com/github/shannonfernandes25/bioinformatics-BPRI-Bioinformatics/blob/main/Docking_9IM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install Biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.1 MB/s eta 0:00:00


In [ ]:
! pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 18.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import requests           # Allows sending HTTP requests (e.g., downloading data from online databases such as PDB, UniProt, or PubChem)
from Bio.PDB import PDBList
from rdkit import Chem                         # Core RDKit module for handling molecules, reading/writing chemical formats
from rdkit.Chem import Descriptors, AllChem    # Descriptors: calculate molecular properties;
                                               # AllChem: advanced chemistry functions (e.g., conformer generation, fingerprints)
import joblib   # Used for saving/loading Python objects (e.g., trained ML models)
                # and parallel processing for speeding up computations

In [ ]:
! pip install chembl-webresource-client   # Installs the ChEMBL client to query the ChEMBL bioactivity database programmatically

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
from chembl_webresource_client.new_client import new_client

# Initialize ChEMBL client
target_client = new_client.target
activity_client = new_client.activity

# Step 1: Search for the target by keyword
search_term = "Leukemia drugs"
targets = target_client.search(search_term)

# Display matching targets
print("Available targets for the search term:")
for target in targets:
    print(f"Target: {target['pref_name']}, ChEMBL ID: {target['target_chembl_id']}")

# Step 2: Choose a target (e.g., the first match)
target_id = targets[0]['target_chembl_id']  # Automatically use the first result
print(f"Selected Target: {targets[0]['pref_name']} ({target_id})")

# Step 3: Fetch inhibitors for the selected target
activities = activity_client.filter(target_chembl_id=target_id, standard_type="IC50").only(
    ["molecule_chembl_id", "canonical_smiles", "standard_value"]
)

# Convert to a Pandas DataFrame
data = []
for activity in activities:
    smiles = activity.get("canonical_smiles")
    ic50 = activity.get("standard_value")
    molecule_id = activity.get("molecule_chembl_id")
    if smiles and ic50:            #Skips entries that don’t have SMILES or IC50 values.
        data.append({"Molecule ChEMBL ID": molecule_id, "SMILES": smiles, "IC50 (nM)": ic50})

df = pd.DataFrame(data)

# Step 4: Save the data to a CSV file
output_file = "Leukemia_compounds.csv"
df.to_csv(output_file, index=False)
print(f"Inhibitors saved to {output_file}")

# Display the first few inhibitors
print(df.head())

Available targets for the search term:
Target: Pleiotropic ABC efflux transporter of multiple drugs, ChEMBL ID: CHEMBL1697658
Target: Myeloid leukemia factor 2, ChEMBL ID: CHEMBL4295830
Target: Leukemia cell, ChEMBL ID: CHEMBL614844
Target: Murine leukemia virus, ChEMBL ID: CHEMBL613746
Target: Feline leukemia virus, ChEMBL ID: CHEMBL612745
Target: Sodium-dependent phosphate transporter 1, ChEMBL ID: CHEMBL4295909
Target: Histone acetyltransferase KAT6A, ChEMBL ID: CHEMBL4523382
Target: Friend leukemia integration 1 transcription factor, ChEMBL ID: CHEMBL5465299
Target: Thrombopoietin receptor, ChEMBL ID: CHEMBL1864
Target: Cytochrome P450 2D6, ChEMBL ID: CHEMBL289
Target: Moloney murine leukemia virus, ChEMBL ID: CHEMBL613500
Target: Rauscher murine leukemia virus, ChEMBL ID: CHEMBL613748
Target: Friend leukemia cell line, ChEMBL ID: CHEMBL614499
Target: Leukemia 60 cell line, ChEMBL ID: CHEMBL614014
Target: Induced myeloid leukemia cell differentiation protein Mcl-1 homolog, ChEMBL I

In [ ]:
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect   # Generates circular Morgan fingerprints as fixed-length bit vectors

In [ ]:
!apt-get install -y autodock-vina

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  autodock autogrid
The following NEW packages will be installed:
  autodock-vina
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 282 kB of archives.
After this operation, 714 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 autodock-vina amd64 1.2.3-2 [282 kB]
Fetched 282 kB in 1s (333 kB/s)
Selecting previously unselected package autodock-vina.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../autodock-vina_1.2.3-2_amd64.deb ...
Unpacking autodock-vina (1.2.3-2) ...
Setting up autodock-vina (1.2.3-2) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
! vina --version

AutoDock Vina v1.2.3


In [ ]:
!apt-get update
!apt-get install -y openbabel

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,415 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packag

In [ ]:
Leukemia_df = pd.read_csv("/content/Leukemia_compounds.csv")
Leukemia_df

,Molecule ChEMBL ID,SMILES,IC50 (nM)
0,CHEMBL1688548,NCCCCN1c2ccccc2Sc2ccc(Cl)cc21,700.0
1,CHEMBL1688549,NCCCCN1c2ccccc2Sc2ccc(C(F)(F)F)cc21,1300.0
2,CHEMBL479,CSc1ccc2c(c1)N(CCC1CCCCN1C)c1ccccc1S2,1300.0
3,CHEMBL567,OCCN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1,1400.0
4,CHEMBL422,CN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1,1400.0
5,CHEMBL726,OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1,1700.0
6,CHEMBL728,CN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1,1700.0
7,CHEMBL1378,CCSc1ccc2c(c1)N(CCCN1CCN(C)CC1)c1ccccc1S2,2300.0
8,CHEMBL1688550,NCCCCN1c2ccccc2Sc2ccccc21,3100.0
9,CHEMBL824,NCCCN1c2ccccc2Sc2ccc(Cl)cc21,3200.0


In [ ]:
# Step 2: Prepare ligands with energy minimization
def prepare_ligands(ligands_csv, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    ligands = pd.read_csv(ligands_csv)
    for index, row in ligands.iterrows():
        smiles = row['SMILES']
        ligand_name = f"ligand_{index + 1}"
        pdb_file = os.path.join(output_dir, f"{ligand_name}.pdb")
        pdbqt_file = os.path.join(output_dir, f"{ligand_name}.pdbqt")

        # Generate 3D structure using RDKit
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)              #add polar hydrogen
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(mol)  # Energy minimization
        Chem.MolToPDBFile(mol, pdb_file)

        # Convert PDB to PDBQT using Open Babel
        os.system(f"obabel {pdb_file} -O {pdbqt_file} --gen3d")
    print("Ligands prepared and saved in PDBQT format.")

# Execute workflow
if __name__ == "__main__":
    #Prepare ligands
    ligands_csv = "/content/Leukemia_compounds.csv"
    output_dir = "pdbqt_ligands"
    prepare_ligands(ligands_csv, output_dir)

Ligands prepared and saved in PDBQT format.


In [ ]:
import Bio

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 9im4.pdb to 9im4.pdb


In [ ]:
import os

pdb_file = list(uploaded.keys())[0]
temp_pdbqt = pdb_file.replace('.pdb', '_temp.pdbqt')
final_pdbqt = pdb_file.replace('.pdb', '_receptor.pdbqt')

# Convert to PDBQT
!obabel "{pdb_file}" -O "{temp_pdbqt}"

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 9im4.pdb)

1 molecule converted


In [ ]:
# Remove invalid tags like ROOT, BRANCH, TORSDOF
bad_tags = {"ROOT", "ENDROOT", "BRANCH", "ENDBRANCH", "TORSDOF"}

with open(temp_pdbqt) as f_in, open(final_pdbqt, "w") as f_out:
    for line in f_in:
        if not any(tag in line for tag in bad_tags):
            f_out.write(line)

print(f"✅ Cleaned receptor saved as: {final_pdbqt}")

✅ Cleaned receptor saved as: 9im4_receptor.pdbqt


In [ ]:
!apt-get update -qq   # Update the package list quietly (-qq = less output)
!apt-get install -y build-essential git unzip   # Install compiler tools, Git, and unzip without asking (-y = auto yes)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
git is already the newest version (1:2.34.1-1ubuntu1.15).
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [ ]:
!git clone https://github.com/Discngine/fpocket.git
!cd fpocket && make

Cloning into 'fpocket'...
remote: Enumerating objects: 11328, done.
remote: Counting objects: 100% (5164/5164), done.
remote: Compressing objects: 100% (1389/1389), done.
remote: Total 11328 (delta 3902), reused 4579 (delta 3766), pack-reused 6164 (from 1)
Receiving objects: 100% (11328/11328), 127.93 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (7351/7351), done.
Updating files: 100% (3895/3895), done.
cd src/qhull/ && make
make[1]: Entering directory '/content/fpocket/src/qhull'
mkdir -p bin lib
gcc -c -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Wwrite-strings -Wshadow -o src/rbox/rbox.o src/rbox/rbox.c
gcc -c -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Wwrite-strings -Wshadow -o src/libqhullstatic/global.o src/libqhull/global.c
gcc -c -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Wwrite-strings -Wshadow -o src/libqhullstatic/stat.o src/libqhull/stat.c
gcc -c -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Wwrite-strings -Wshadow -o src/libqhullstatic/geom2.o src/lib

In [ ]:
import os
os.environ["PATH"] += "/content/fpocket/bin"

In [ ]:
!/content/fpocket/bin/fpocket -f 9im4.pdb

***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


In [ ]:
import numpy as np

def get_center_from_pdb(pocket_file):
    coords = []
    with open(pocket_file) as f:
        for line in f:
            if line.startswith("ATOM"):
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                coords.append([x, y, z])
    coords = np.array(coords)
    return coords.mean(axis=0)

center = get_center_from_pdb("/content/9im4_out/pockets/pocket10_atm.pdb")
print(f"Suggested Vina Box Center:\n  --center_x {center[0]:.2f} --center_y {center[1]:.2f} --center_z {center[2]:.2f}")


Suggested Vina Box Center:
  --center_x -40.99 --center_y 16.90 --center_z 15.02


In [ ]:
center_1 = get_center_from_pdb("/content/9im4_out/pockets/pocket11_atm.pdb")
print(f"Suggested Vina Box Center:\n  --center_x {center_1[0]:.2f} --center_y {center_1[1]:.2f} --center_z {center_1[2]:.2f}")

Suggested Vina Box Center:
  --center_x -21.72 --center_y 24.16 --center_z -29.09


In [ ]:
type(center_1)

numpy.ndarray

In [ ]:
center_2 = get_center_from_pdb("/content/9im4_out/pockets/pocket1_atm.pdb")
print(f"Suggested Vina Box Center:\n  --center_x {center_2[0]:.4f} --center_y {center_2[1]:.4f} --center_z {center_2[2]:.4f}")

Suggested Vina Box Center:
  --center_x -53.5503 --center_y 36.0111 --center_z -5.1268


In [ ]:
docking_results_dir = "docking_results"  # Directory to save results
box_center = [-21.72, 24.16, -29.09]  # Center of docking box (x, y, z)
box_size = [40, 40, 40]  # Size of docking box (x, y, z)

In [ ]:
exhaustiveness = 3  # Increase to explore more conformations  # it must be at least 8

# Step : Perform docking with refined parameters
def dock_ligands(protein_pdbqt, ligands_dir, output_dir, box_center, box_size, exhaustiveness):
    os.makedirs(output_dir, exist_ok=True)
    ligand_files = [f for f in os.listdir(ligands_dir) if f.endswith(".pdbqt")]

    print(f"Found {len(ligand_files)} ligands for docking.")

    for ligand_file in ligand_files:
        ligand_path = os.path.join(ligands_dir, ligand_file)
        output_path = os.path.join(output_dir, f"{ligand_file}_docked.pdbqt")

        docking_command = (
            f"vina --receptor {protein_pdbqt} --ligand {ligand_path} --out {output_path} "
            f"--center_x {box_center[0]} --center_y {box_center[1]} --center_z {box_center[2]} "
            f"--size_x {box_size[0]} --size_y {box_size[1]} --size_z {box_size[2]} "
            f"--exhaustiveness {exhaustiveness}"
        )

        os.system(docking_command)
        if os.path.exists(output_path):
            print(f"Docking completed for {ligand_file}")
        else:
            print(f"Docking failed for {ligand_file}")

# Step : Analyze docking results
def analyze_results(docking_results_dir, output_csv):
    results = []
    for pdbqt_file in os.listdir(docking_results_dir):
        if pdbqt_file.endswith("_docked.pdbqt"):
            with open(os.path.join(docking_results_dir, pdbqt_file), "r") as f:
                score = None
                for line in f:
                    if "REMARK VINA RESULT" in line:
                        score = float(line.split()[3])
                        break
                if score is not None:
                    results.append((pdbqt_file, score))

    results.sort(key=lambda x: x[1])  # Sort by docking score (lower is better)

    # Save results to CSV
    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Ligand File", "Docking Score (kcal/mol)"])
        writer.writerows(results)

    print(f"Docking scores saved to {output_csv}")
    print("Top docking results:")
    for result in results[:10]:
        print(result)

# Execute workflow
if __name__ == "__main__":

    # Step : Perform docking
    dock_ligands("/content/9im4_receptor.pdbqt", "/content/pdbqt_ligands", docking_results_dir, box_center, box_size, exhaustiveness)

    # Step : Analyze results
    analyze_results(docking_results_dir, "Shannon_docking_scores.csv")

Found 23 ligands for docking.
Docking completed for ligand_9.pdbqt
Docking completed for ligand_2.pdbqt
Docking completed for ligand_17.pdbqt
Docking completed for ligand_18.pdbqt
Docking completed for ligand_16.pdbqt
Docking completed for ligand_23.pdbqt
Docking completed for ligand_12.pdbqt
Docking completed for ligand_22.pdbqt
Docking completed for ligand_7.pdbqt
Docking completed for ligand_3.pdbqt
Docking completed for ligand_5.pdbqt
Docking completed for ligand_15.pdbqt
Docking completed for ligand_20.pdbqt
Docking completed for ligand_14.pdbqt
Docking completed for ligand_13.pdbqt
Docking completed for ligand_11.pdbqt
Docking completed for ligand_10.pdbqt
Docking completed for ligand_21.pdbqt
Docking completed for ligand_8.pdbqt
Docking completed for ligand_6.pdbqt
Docking completed for ligand_4.pdbqt
Docking completed for ligand_1.pdbqt
Docking completed for ligand_19.pdbqt
Docking scores saved to Shannon_docking_scores.csv
Top docking results:
('ligand_22.pdbqt_docked.pdbqt', 